<a href="https://colab.research.google.com/github/jshin13/coursework/blob/temp/Lab2_Decision_trees_jshin69.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 2: Decision trees - The CART algorithm

In this lab, you will be programming your own decision tree CART algorithm with Gini criterion. 

In the following lines, complete the functions replacing the comments # YOUR CODE GOES HERE or the variables equal to 'None'  with your code.

To evaluate the algorithm we will be working with the [Early stage diabetes risk prediction dataset](https://archive.ics.uci.edu/ml/datasets/Early+stage+diabetes+risk+prediction+dataset.) containing predictor variables about patients with diabetes and controls. 


**0. Moving the lab to your folder**

Copy this notebook to your personal directory

In [614]:
# Library import. Feel free to add more libraries that you may use in your code

import urllib.request
import pandas as pd
from pandas.api.types import is_numeric_dtype
import numpy as np

**1. Data Loading**

In [615]:
sData='http://archive.ics.uci.edu/ml/machine-learning-databases/00529/diabetes_data_upload.csv'
urllib.request.urlretrieve(sData,'./datafile.csv') # The data is stored in your drive folder under the name 'datafile.csv'
df=pd.read_csv('./datafile.csv')

If you print the head of the dataframe, you can observe the different predictor variables (Age, Gender, Polyuria, etc) and the labels of the first five observations

In [616]:
df.head()

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,class
0,40,Male,No,Yes,No,Yes,No,No,No,Yes,No,Yes,No,Yes,Yes,Yes,Positive
1,58,Male,No,No,No,Yes,No,No,Yes,No,No,No,Yes,No,Yes,No,Positive
2,41,Male,Yes,No,No,Yes,Yes,No,No,Yes,No,Yes,No,Yes,Yes,No,Positive
3,45,Male,No,No,Yes,Yes,Yes,Yes,No,Yes,No,Yes,No,No,No,No,Positive
4,60,Male,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Positive


In the next cell, you can print a single observation by selecting its correspondent series using iloc.

In [617]:
series=df.iloc[3,:-1] # The last column is removed (it's the class)
print(series)

Age                     45
Gender                Male
Polyuria                No
Polydipsia              No
sudden weight loss     Yes
weakness               Yes
Polyphagia             Yes
Genital thrush         Yes
visual blurring         No
Itching                Yes
Irritability            No
delayed healing        Yes
partial paresis         No
muscle stiffness        No
Alopecia                No
Obesity                 No
Name: 3, dtype: object


**2. Dataset descriptors**
**Data and notation used in the algorithm:**
X is the training subset containing N observations
Each observation, x_n is a vector containing the predictor variables (size M) for each observation.
y_n is the label of x_n
J: total number of classes
N: total number of observations (number of training vectors)
M: number of predictor variables (feature vector length)

**TASK 1:** In the next cell, calculate the N, M and J.

In [618]:
#First steps: Determine the number of features, classes and observations 
#(replace 'None' with your code) (5 POINTS)
y=df['class']
N=df.shape[0]
M=series.shape[0]
J=len(df['class'].unique())

print('Number of observations: '+str(N))
print('Number of predictor variables: '+str(M))
print('Number of classes: '+str(J))

Number of observations: 520
Number of predictor variables: 16
Number of classes: 2


Expected output:

Number of observations: 520

Number of predictor variables: 16

Number of classes: 2


**TASK 2:** Split the data into, approx, 80% training and 20% testing (you can choose the first 80% observations as training and the rest as testing)

In [619]:
# Split the data into, approx, 80% training and 20% testing
#(replace 'None' with your code)  (5 POINTS)
df = df.sample(frac=1)
dfTrain=df[:round(M*0.8)]
dfTest=df[round(M*0.8):]
print(dfTrain.shape[0] + dfTest.shape[0])

520


# 3. The CART Algorithm
*1.	Find each predictor’s best split:*

* a.	Determine if the predictor is categorical or numerical


* b.	For each predictor, obtain unique values. 

* c.	For each predictor: using the unique values, go through each value to examine each predictor’s possible split point


* - i.	The best split point (threshold) is the one that maximizes the splitting criterion1 (impurity gain) the most.

* - ii.	In categorical predictors, we will have to consider a possible node for each category.


*2.	Find the predictor that provides the best split:*

* a.	Select the predictor that maximizes the splitting criterion (the one that produces the highest impurity gain.


*3.	Split the data X into two new nodes using the split point calculated in point 1 for the predictor selected in point 2.*


*4.	Repeat point 1 for each branch if stopping rules are not satisfied:*

* a.	Stopping rules: 
* * i.	Maximum tree depth (maximum number of nodes in a row)
* * ii.	All the remaining training observations in a resulting branch belong to a single class (maximum purity)
* * iii.	All observations in a node have the same predictor’s values: it is not possible to split more. 
* * iv.	The node size is less than the minimum node size (number of observations in a node)

In the following cells, create the functions required for each point.


**TASK 3:** 1.a.  Determine if the predictor is categorical or numerical

In [620]:
# 1.a Is the class numerical? (5 POINTS)

def is_numerical(df):
    """Test if values in a series are numeric and returns a vector with boolean results"""
    #YOUR CODE GOES HERE
    return dfTrain.iloc[0,:].astype(str).str.isnumeric().tolist()

In [621]:
# Example
is_numerical(dfTrain)

[True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

Expected output: 


[True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]



**TASK 4:** 1.b. b.	For each predictor, obtain unique values (categories)

In [622]:
#1.b Function to obtain categories per predictor (5 POINTS)
def unique_values(df,column):
  """
  This function returns a list with all the possible categories present in a predictor variable
  Inputs: dataframe (df) and string containing the name of the predictor variable (column)
  """
    #YOUR CODE GOES HERE
  return df[column].unique().tolist()


In [623]:
#Example
categories=unique_values(df,'Gender')
print(categories)
print(categories[0])

['Male', 'Female']
Male


Expected output:

['Male' 'Female']

Male

**TASK 5:** 1.c.	For each predictor: using the unique values, go through each value to examine each predictor’s possible split point

1. c. i. Search for the best split point (threshold) is the one that maximizes the splitting criterion1 (impurity gain) the most.

In [624]:
#1.c.i Generate the gini function that calculates impurity  (15 POINTS)
def impurity_gini(df):
  """
  Calculates and returns the Gini impurity of the input dataframe (df)
  """
  #YOUR CODE GOES HERE
  temp = []
  for i in unique_values(df,'class'):
      temp.append((df['class'] == i).sum() / df.shape[0])  
  return 1 - sum(np.array(temp)**2)

def impurity_gain(df, dfl, dfr):
  """
  Calculates and returns the Impurity Gain considering that df is used to
  calculate the initial impurity (or node t), and dfl and dfr to calculate the
  weights and impurities of node L and R, respectively
  """
  #YOUR CODE GOES HERE
  return impurity_gini(df) - dfl.shape[0]/df.shape[0] * impurity_gini(dfl) - dfr.shape[0]/df.shape[0] * impurity_gini(dfr)  


In [625]:
impurity_gini(df)

0.47337278106508873

Expected output: 0.47337278106508873

In [626]:
impurity_gain(df,df,df)

-0.47337278106508873

Expected output: -0.47337278106508873

**TASK 6:** Search for the split point (or threshold) that provides the highest impurity gain in numerical predictor variables (for instance, 'Age' is a numerical variable).

In [627]:
#1.c.i Numerical split analysis (10 POINTS)
def numerical_max_search(df, column):
  """ 
  Returns the max impurity gain (ig_max) of the variable defined by 'column' in the df subset.
  It also returns the value of the split point (category_max) that yields the 
  highest impurity gain for that variable.
  """
  #YOUR CODE GOES HERE
  ig_max = 0
  category_max = 0

  for i in unique_values(df,column):
      dfl = df[df[column] <= i]
      dfr = df[df[column] > i]
      temp = impurity_gain(df,dfl,dfr)

      if ig_max < temp:
        ig_max = temp
        category_max = i

  return ig_max, category_max

In [628]:
numerical_max_search(df, 'Age')

(0.015152664201645794, 34)

Expected Output:

(0.015152664201645794, 34)

**TASK 7:** 1.c.	For each predictor: using the unique values, go through each value to examine each predictor’s possible split point

1. c. ii.	In categorical predictors, we will have to consider a possible node for each category. This means that we have to perform a search of best split for each category. In the dataset that we are using, each of the categorical variables have only two possible categories (Yes or No - Male or Female). However, a code that consider more than 2 categories would be desirable.

In [629]:
# 1.c.ii Categorical split analysis (10 POINTS)
def categorical_max_search(df, column):
    """ 
    Returns the max impurity gain (ig_max) of all categories in the variable defined by column
    it also returns the name of the category that yields the highest impurity gain (category_max).
    """
    #YOUR CODE GOES HERE
    ig_max = 0
    category_max = None

#   if ~np.array(is_numerical(df[column])):
    for i in unique_values(df, column):
        dfl = df[df[column] == i]
        dfr = df[df[column] != i]
        temp = impurity_gain(df, dfl, dfr)

        if ig_max < temp:
            ig_max = temp
            category_max = i

    return ig_max, category_max

In [630]:
categorical_max_search(df, 'Polydipsia')

(0.19922151623026904, 'No')

Expected output: (0.19922151623026907, 'No')

**TASK 8:** 2.	Using the previous functions, find the predictor that provides the best split: 
a.	Select the predictor that maximizes the splitting criterion (the one that produces the highest impurity gain).


In [631]:
# 2.a Find predictor that provides best split (10 POINTS)
def find_max_predictor(df):
  """
  This function finds the predictor variable (Max_predictor), category or threshold
  (Max_category) and their associated impurity gain (Max_ig) in the input dataset (df)
  using the previous functions 
  """
  #YOUR CODE GOES HERE
  Max_ig = 0
  Max_predictor = None
  Max_category = None

  for idx, i in enumerate(is_numerical(df)[:-1]): #exclude label from the list of predictor     
      if i:
        temp_ig, temp_cat = numerical_max_search(df, df.columns[idx]) # if numerical, do numerical search
      else:
        temp_ig, temp_cat = categorical_max_search(df, df.columns[idx]) # if not numerical, do categorical search 

      if Max_ig < temp_ig: # if better impurity gain
          Max_ig = temp_ig # update max impurity gain
          Max_predictor = df.columns[idx] # update max predictor
          Max_category = temp_cat # update max category

  return Max_ig, Max_predictor, Max_category

In [632]:
# find_max_predictor(dfTrain)
# numerical_max_search(dfTrain,'Age')
# impurity_gini(dfTrain)
# dfTrain

In [633]:
# Example
Max_ig, Max_predictor, Max_category=find_max_predictor(df)
print('Maximum Impurity Gain: ' +str(Max_ig) +' obtained with the predictor '+ Max_predictor +' and category ' +str(Max_category))

Maximum Impurity Gain: 0.20991841189440502 obtained with the predictor Polyuria and category No


Expected output: 

Maximum Impurity Gain: 0.20991841189440502 obtained with the predictor Polyuria and category No

**TASK 9:** 3.	Split any input data X (a dataframe) into two new nodes using the 
split point and predictor selected obtained with find_max_predictor.

In [634]:
# 3. Split data (15 POINTS)
def split_data(df):
    """
    Splits the input subset (df) into two smaller subsets (dfL and dfR) that define
    the split that provides the highest Impurity Gain. It also returns the predictor
    (Max_predictor) and category (Max_category) that favor that split. The code might 
    be different for numerical and categorical predictor variables.
    """

    #YOUR CODE GOES HERE
    _, Max_predictor, Max_category = find_max_predictor(df)

    dfL = df[df[Max_predictor] == Max_category]
    dfR = df[df[Max_predictor] != Max_category]
  
    return dfL,dfR, Max_predictor, Max_category 

In [635]:
# This defines the first node of the tree for the df dataset
dfL,dfR,predictor,treshold=split_data(df)
# This calculates the Impurity Gain when going from the initial dataset (df) to 
# the left and right subsets
impurity_gain(df,dfL,dfR)

0.20991841189440502

Expected output:
0.20991841189440502

**Additional code:**
The following function and classes serve to store information of the nodes and leaves. You do not need to code anything here. :)

In [636]:
def classify_leaf(df):
  """ 
  Counts the number of observations per class (class probability) in the input df 
  This function serves to create the classification info stored in the leaf.
  """

  classes=df.iloc[:,-1] # Labels in df
  classes_uni=classes.unique() # this variable contains the classes that exist in df
  result={Tclass:(sum(classes==Tclass)/len(df)) for Tclass in classes_uni} #proportion of observations per class
  return result

class tree_leaf:
    """
    This class will contain the results of the leaf (ex: Positive: 0.8, Negative:0.2)
    """

    def __init__(self, df):
        self.predictions = classify_leaf(df)

class tree_node:
    """
    This class will contain the predictor and treshold of the node and a the two 
    nested nodes (left and right)
    """

    def __init__(self,
                 predictor,
                 threshold,
                 left_branch,
                 right_branch):
        self.predictor = predictor
        self.threshold = threshold
        self.left_branch = left_branch
        self.right_branch = right_branch

**TASK 10:** Stopping rules. These rules will serve to decide when to stop looking for a node and create a leaf. Conventional algorithms might have more stopping rules, but we will restrict our rules to only four conditions. You will have to code the functions that check these rules.

4.a.	Stopping rules:

i.	Maximum tree depth (maximum number of nodes in a row) (this will be checked in the main function)

ii.	All the remaining training observations in a resulting branch belong to a single class (maximum purity)

iii.	All observations in a node have the same predictor’s values: it is not possible to split more. 

iv.	The node size is less than the minimum node size (number of observations in a node)


In [637]:
impurity_gini(df)

0.47337278106508873

In [638]:
# check stopping rules (10 POINTS)

#4.a.ii
def check_impurity(df):
  #returns TRUE if impurity of the data in df is equal to 0.
 
    #YOUR CODE GOES HERE
    if impurity_gini(df) == 0: return True
    return False

#4.a.iii
def check_predictor(df):
  #Checks if all the observations have exactly the same variable vectors 
  #(same values for every single predictor variable)
    
    #YOUR CODE GOES HERE
    for i in df.columns:
        if len(unique_values(df, i)) == 1:
            pass
        else: return False
    return True        
  
#4.a.iv
def check_node_length(df, min_partition_length):
  #returns TRUE if the number of observations in df is less than min_partition_length
  
    #YOUR CODE GOES HERE (you have finished!!)
    if df.shape[0] < min_partition_length:
        return True
    return False


**End of the lab: Train the tree**

If all your functions have been coded as requested, the following functions should train and evaluate a decision tree. 

**TASK 11:** Code the training of the right (false) branch

In [639]:
# train the tree  (10 POINTS)
def train_tree_mlma(df, tree_depth=0, min_partition_length=2, max_tree_depth=200):

    dfL,dfR,predictor,threshold=split_data(df)
    tree_depth+=1 #the tree depth in this path is extended
    print('Tree depth: ' +str(tree_depth))
    #tree_depth==max_tree_depth checks condition 4.a.i
    #Left branch (true)
    if check_impurity(dfL) or check_predictor(dfL) or check_node_length(dfL,min_partition_length) or tree_depth==max_tree_depth:
        left_node=tree_leaf(dfL)
    else:
        left_node=train_tree_mlma(dfL, tree_depth)
  
    #Right branch (false)

    # YOUR CODE GOES HERE

    if check_impurity(dfR) or check_predictor(dfR) or check_node_length(dfR,min_partition_length) or tree_depth==max_tree_depth:
        right_node=tree_leaf(dfR)
    else:
        right_node=train_tree_mlma(dfR, tree_depth)

    return tree_node(predictor, threshold, left_node, right_node)

In [640]:
my_tree=train_tree_mlma(dfTrain)

Tree depth: 1
Tree depth: 2


In [641]:
def print_tree_mlma(my_tree, jump=''):
  jump=jump+'-'
  if isinstance(my_tree, tree_leaf):
    print(jump+'Result: ')
    print(my_tree.predictions)
  else:
    if is_numeric_dtype(my_tree.threshold):
      print(jump + '> If ' + str(my_tree.predictor) + ' <= ' + str(my_tree.threshold))
      print_tree_mlma(my_tree.left_branch,jump)
      print(jump+'> Else: ')
      print_tree_mlma(my_tree.right_branch,jump)
    
    else:
      print(jump + '> If ' + str(my_tree.predictor) + ' is ' + str(my_tree.threshold))
      print_tree_mlma(my_tree.left_branch,jump)
      print(jump+'> Else: ')
      print_tree_mlma(my_tree.right_branch,jump)


In [642]:
print_tree_mlma(my_tree)

-> If Polydipsia is No
--> If Irritability is No
---Result: 
{'Negative': 1.0}
--> Else: 
---Result: 
{'Positive': 1.0}
-> Else: 
--Result: 
{'Positive': 1.0}


In [643]:
my_series=dfTest.iloc[56,:]
predictionprediction=classify_tree_mlma(my_tree, my_series )
print('True class: ' + my_series[-1] + ' - ' + my_series.keys()[-1])
print('Predicted class: ')
print(prediction)

True class: Positive - class
Predicted class: 
{'Positive': 1.0}


In [644]:
def classify_tree_mlma(my_tree, my_series):
  """
  Classifies the input series and returns the predictions
  """
  if isinstance(my_tree, tree_leaf):
    return my_tree.predictions
  else:
    if is_numeric_dtype(my_tree.threshold): # Numeric
      if my_series[my_tree.predictor] <= my_tree.threshold:
        return classify_tree_mlma(my_tree.left_branch,my_series)
      else:
        return classify_tree_mlma(my_tree.right_branch,my_series)
    else: #Categorical
      if my_series[my_tree.predictor] == my_tree.threshold:
        return classify_tree_mlma(my_tree.left_branch,my_series)
      else:
        return classify_tree_mlma(my_tree.right_branch,my_series)

In [645]:
my_series=dfTest.iloc[33,:]
prediction=classify_tree_mlma(my_tree, my_series)
print('True class: ' + my_series[-1] + ' - ' + my_series.keys()[-1])
print('Predicted class: ')
print(prediction)

True class: Positive - class
Predicted class: 
{'Positive': 1.0}


**BONUS:** 
Code a function that calculates the accuracy of the trained tree when classifying the whole dfTest subset.

In [646]:
# YOUR CODE GOES HERE (+20 POINTS)
pred = []
y = []

N = dfTest.shape[0]

for i in range(N):
    my_series=dfTest.iloc[i,:]
    pred = pred + [i for i in classify_tree_mlma(my_tree, my_series).keys()]
    y.append(my_series['class'])

acc = sum(np.array(pred) == np.array(dfTest['class'].tolist())) / N
acc2 = sum(np.array(pred) == np.array(y)) / N

In [647]:
print(f'Accuracy is {round(acc * 100, 2)} %')

Accuracy is 82.64 %


**You are ready to submit in Blackboard!**

Please suffix your colab file with _<jhID> 
o	eg: Lab2_Decision_trees_myjhID12

